In [1]:
from __future__ import print_function

import numpy as np
import pandas as pd

# GUI library
import panel as pn
import panel.widgets as pnw

# Chart libraries
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Legend, HoverTool


from bokeh.palettes import Spectral5, Set2, Category20
from bokeh.events import SelectionGeometry

# Dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler, LabelEncoder
from umap import UMAP

#
from shapely.geometry import MultiPoint, MultiLineString, Polygon, MultiPolygon, LineString
from shapely.ops import unary_union
from shapely.ops import triangulate

# local scripts
from Embedding.Rangeset import Rangeset

pn.extension()

# Parameters

In [2]:
dataset_name = 'Thermo'

bins = 5

show_labels = True
labels_column = 'Component Name'

overview_height = 700

histogram_width = 250
show_numpy_histogram = True

rangeset_threshold = 1

# Load data

In [3]:
solutes_df = pd.read_excel("./data/Data_SD.xlsx", sheet_name="Solutes")
df = solutes_df

df2 = pd.read_excel("./data/20201109_Descriptors_nored.xlsx")
df = df.join(df2.set_index('DDB No'), on='DDB ID')
top20_classes = df['Übergruppe (ÜG)'].value_counts().index.tolist()[:20]
groups = pd.concat([df['Übergruppe (ÜG)'],
                       df['Übergruppe (ÜG).1'].rename({'Übergruppe (ÜG).1': 'Übergruppe'})], axis=0)
groups.value_counts().head(20).sort_index()
top20_groups = groups.value_counts().head(20).index.tolist()

df_raw

df = df_raw.dropna().copy()

label_encoders = {}

for var in ['species_short', 'island', 'sex']:
    label_encoders[var] = LabelEncoder().fit(df[var])
    df.loc[:,var] = label_encoders[var].transform(df[var]) + 1

import seaborn as sns

sns.heatmap(df.corr(), vmin=-1, vmax=1, cmap='RdBu')

# Preprocessing

In [4]:
# attributes to be included in the overlays
selected_var = ['u1','u2','u3','u4','Molar Mass','Relative Polarity','H-Bond Acceptors','H-Bond Donors']
group_var = ['Primary Group','All Groups']

# slider range and step size
# {'variable_name': (min,max,stepsize)}
custom_range = {}

# default values for slider min/max
# {'variable_name': (min,max)}
default_range = {'u1': (-0.98,0.92),
                 'u2': (-.83,1.52),
                 'u3': (-3,.56),
                 'u4': (-.59,1.81),
                'Molar Mass': (18,204),
                'Relative Polarity': (0,0.59)}

In [36]:
# which variables to use for the embedding
selected_var_embd = selected_var[:4]

# set up embedding
#embedding = PCA(n_components=2)
embedding = MDS(n_components=2, random_state=42)
#embedding = UMAP(random_state=42)

scaler = StandardScaler()
df2 = df.loc[:,selected_var_embd].copy()
df2 = df2.append(df2.loc[:,selected_var_embd].median(), ignore_index=True)

for var in selected_var_embd:
    medians = df.loc[:,selected_var_embd].median().copy()
    medians[var] = df[var].max()
    df2 = df2.append(medians, ignore_index=True)
    
for var in selected_var_embd:
    medians = df.loc[:,selected_var_embd].median().copy()
    medians[var] = df[var].min()
    df2 = df2.append(medians, ignore_index=True)
    
X_scaled = scaler.fit_transform(df2)#[selected_var_embd])

# some projections change the original data, so we make a copy
# this can cost a lot of memory for large data
X  = X_scaled.copy()
pp = embedding.fit_transform(X)

In [37]:
x_range = pp[:,0].max() - pp[:,0].min()
y_range = pp[:,1].max() - pp[:,1].min()

# keep the aspect ration of the projected data
overview_width = min(1000,int(overview_height * x_range / y_range))
histogram_height = max(200,int(histogram_width * y_range / x_range))

In [39]:
from Embedding.ProjectionQuality import projection_quality

df['projection quality'] = projection_quality(X_scaled, pp)[:-(len(selected_var_embd)*2+1)]
selected_var += ['projection quality']

print('mean projection quality', df['projection quality'].mean())

mean projection quality 0.4367884878407749


In [40]:
rangeset = Rangeset(pp, df)
rangeset.threshold = rangeset_threshold
rangeset.size_inside = 3
rangeset.size_outside = 6

In [41]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from sklearn.metrics import pairwise_distances

D = csr_matrix(pairwise_distances(pp))
MST = minimum_spanning_tree(D)
print('max edge in MST: {:.2f}'.format(MST.A.max()))

max edge in MST: 0.94


In [42]:
eps = np.quantile(MST[MST.nonzero()], .75) + 1.5*(np.quantile(MST[MST.nonzero()], .75) - np.quantile(MST[MST.nonzero()], .25))

In [43]:
print('Wilkinson epsilon: {:.2f}'.format(eps))

Wilkinson epsilon: 0.50


In [44]:
#sorted(MST[MST.nonzero()].A[0])

# GUI

## Vis elements

**overview chart** shows a large version of the embedding

In [47]:
#df['x'] = pp[:,0]
#df['y'] = pp[:,1]
df['x'] = pp[:-(len(selected_var_embd)*2+1),0]
df['y'] = pp[:-(len(selected_var_embd)*2+1),1]

In [49]:
source = ColumnDataSource()
source.data['index'] = df.index

for var in selected_var:
    source.data[var] = df[var]

source.data['x'] = df['x']
source.data['y'] = df['y']

TOOLS = "pan,wheel_zoom,box_zoom,box_select,lasso_select,help,reset,save"
TOOLTIPS = [(v, '@{'+v+'}') for v in source.data.keys()]

overview = figure(width=overview_width, height=overview_height, 
                  active_drag="lasso_select", tools=TOOLS)

r = overview.scatter(source=source, x='x', y='y', color="#333333", muted_alpha=0,
                 size=3, level='underlay', name='points',
                 line_color=None, legend_label='data')

hover_tool = HoverTool(renderers=[r], tooltips=TOOLTIPS)
overview.add_tools(hover_tool)

if labels_column:
    labels_txt = source.data['index'].astype(str) if labels_column == 'index' else df[labels_column].astype(str)
    overview.text(x=source.data['x'], y=source.data['y'], text=labels_txt, legend_label='labels',
                  font_size="5pt", x_offset=5, y_offset=5, muted_alpha=0,
                  text_baseline="middle", text_align="left", color='#666666', level='glyph')

source_selection = ColumnDataSource({'x': [], 'y': []})
overview.patch(source=source_selection, x='x', y='y', fill_color=None, line_width=2, line_color='#333333',
               level='annotation')

overview.legend.location = 'top_left'
overview.legend.label_height=1
overview.legend.click_policy='mute'
overview.legend.visible = True

overview.outline_line_color = None

overview.xgrid.visible = False
overview.ygrid.visible = False
overview.xaxis.visible = False
overview.yaxis.visible = False
overview.toolbar.logo = None

In [50]:
# Check the embedding with the code below
# pn.Row(overview)

**small multiples** charts are created upon request

In [51]:
def _make_chart( var, df_polys, df_scatter, bounds, cnt_in, cnt_out ):
    global df

    xvals = df[var].unique()
    is_categorical = False
    if len(xvals) < 10:
        is_categorical = True
        xvals = sorted(xvals.astype(str))
    
    global histogram_width
    p = figure(width=histogram_width, height=histogram_height, title=var, x_range=overview.x_range, y_range=overview.y_range)
    df_scatter['size'] = df_scatter['size'] * histogram_height / overview_height
    
    p.multi_polygons(source=df_polys, xs='xs', ys='ys', color='color', fill_alpha=0.5, level='image', line_color=None)
    p.scatter(source=df_scatter, x='x', y='y', color='color', size='size', level='overlay')
    
    global source_selection
    p.patch(source=source_selection, x='x', y='y', fill_color=None, level='annotation', line_width=2, line_color='#333333')
    
    p.xgrid.visible = False
    p.ygrid.visible = False
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.toolbar.logo = None
    p.toolbar_location = None
    p.border_fill_color = '#f0f0f0'
    
    p_histo = figure(height=100, width=histogram_width, name='histo')
    if is_categorical:
        p_histo = figure(height=100, width=histogram_width, name='histo', x_range=xvals)
        p_histo.vbar(x=xvals, top=cnt_in, bottom=0, width=0.9, line_color='white', color=rangeset.colormap)
        p_histo.vbar(x=xvals, top=0, bottom=np.array(cnt_out)*-1, width=0.9, line_color='white', color=rangeset.colormap)
    else:
        p_histo.quad(bottom=[0]*len(cnt_in), top=cnt_in, left=bounds[:-1], right=bounds[1:], line_color='white', color=rangeset.colormap)
        p_histo.quad(bottom=np.array(cnt_out)*(-1), top=[0]*len(cnt_out), left=bounds[:-1], right=bounds[1:], line_color='white', color=rangeset.colormap)

        df_select = df[df[var] < bounds[0]]
        p_histo.square(df_select[var], -.5, color=rangeset.colormap[0])
        df_select = df[df[var] > bounds[-1]]
        p_histo.square(df_select[var], -.5, color=rangeset.colormap[-1])
    
    p_histo.toolbar.logo = None
    p_histo.toolbar_location = None
    p_histo.xgrid.visible = False
    p_histo.xaxis.minor_tick_line_color = None
    p_histo.yaxis.minor_tick_line_color = None
    p_histo.outline_line_color = None
    p_histo.border_fill_color = '#f0f0f0'
    
    global show_numpy_histogram
    if show_numpy_histogram:
        if is_categorical:
            frequencies, edges = np.histogram(df[var], bins=len(xvals))
            p_histo.vbar(x=xvals, bottom=0, width=.5, top=frequencies*-1,
                         line_color='white', color='gray', line_alpha=.5,  fill_alpha=0.5)
        else:
            frequencies, edges = np.histogram(df[var])
            p_histo.quad(bottom=[0]*len(frequencies), top=frequencies*-1, left=edges[:-1], right=edges[1:], 
                     line_color='white', color='gray', line_alpha=.5,  fill_alpha=0.5)
    
    return (p, p_histo)

## Create input widget (buttons, sliders, etc) and layout

In [52]:
class MyCheckbox(pnw.Checkbox):
    variable = ""
    
    def __init__(self, variable="", slider=None, **kwds):
        super().__init__(**kwds)
        
        self.variable = variable
        self.slider = slider
        
def init_slider_values(var):
    vmin = df[var].min()
    vmax = df[var].max()
    step = 0
    
    if var in custom_range:
        vmin,vmax,step = custom_range[var]
    value = (vmin,vmax)
    
    if var in default_range:
        value = default_range[var]
        
    return (vmin, vmax, step, value)

Create all toplevel GUI elements

In [53]:
ranges_embd = pn.Column()
ranges_aux = pn.Column()

sliders = {}

def create_slider(var):
    vmin, vmax, step, value = init_slider_values(var) 
    slider = pnw.RangeSlider(name=var, start=vmin, end=vmax, step=step, value=value) 

    checkbox = MyCheckbox(name='', variable=var, value=False, width=20, slider=slider)
    return pn.Row(checkbox,slider)
    
for var in selected_var:
    s = create_slider(var)
    sliders[var] = s

    if var in selected_var_embd:
        ranges_embd.append(s)
    else:
        ranges_aux.append(s)
        
selected_var = []

for r in ranges_embd:
    selected_var.append(r[1].name)
for r in ranges_aux:
    selected_var.append(r[1].name)

In [54]:
gui_colormap = pn.Row(pn.pane.Str(background=rangeset.colormap[0], height=30, width=20), "very low",
                      pn.pane.Str(background=rangeset.colormap[1], height=30, width=20), "low",
                      pn.pane.Str(background=rangeset.colormap[2], height=30, width=20), "medium",
                      pn.pane.Str(background=rangeset.colormap[3], height=30, width=20), "high",
                      pn.pane.Str(background=rangeset.colormap[4], height=30, width=20), "very high", sizing_mode='stretch_width')

selectColoring = pn.widgets.Select(name='', options=['None']+selected_var+group_var)


# set up the GUI
layout = pn.Row(pn.Column(
    pn.Row(pn.pane.Markdown('''# NoLiES: The non-linear embedding surveyor\n
NoLiES augments the projected data with additional information. The following interactions are supported:\n
* **Attribute-based coloring** Chose an attribute from the drop-down menu below the embedding to display contours for different values ranges.
* **Selective muting**: Click on the legend to mute/hide parts of the chart.
* **Contour control** Change the slider ranges to change the contours.
* **Histograms** Select the check-box next to the slider to view the attribute's histogram.
* **Selection** Use the selection tool to outline a set of points and share this outline across plots.''', sizing_mode='stretch_width'), 
           margin=(0, 25,0,25)),
    pn.Row(
        pn.Column(pn.pane.Markdown('''# Attributes\nEnable histograms with the checkboxes.'''), 
                  '## Embedding',
                  ranges_embd,
                  #pn.layout.Divider(),
                  '## Auxiliary',
                  ranges_aux, margin=(0, 25, 0, 0)),
        pn.Column(pn.pane.Markdown('''# Embedding - '''+type(embedding).__name__+'''&nbsp;&nbsp;  Dataset - '''+dataset_name, sizing_mode='stretch_width'), 
                  overview, 
                  pn.Row(selectColoring, gui_colormap)
                 ), 
        margin=(0,25,25,25)
        )), 
    #pn.Row(sizing_mode='stretch_height'), 
    #pn.Row(pn.pane.Markdown('''Data source: Gorman KB, Williams TD, Fraser WR (2014) Ecological Sexual Dimorphism and Environmental Variability within a Community of Antarctic Penguins (Genus Pygoscelis). PLoS ONE 9(3): e90081. doi:10.1371/journal.pone.0090081''',
    #      width=800), sizing_mode='stretch_width', margin=(0,25,0,25))),
    pn.GridBox(ncols=3, sizing_mode='stretch_both', margin=(25,25,0,0)),
    background='#efefef'
    )

Adjust the order of the variable so that it reflects the sorting of the range sliders (we distinguish between those used for embedding and auxiliary ones).

In [55]:
# Check the GUI with the following code - this version is not interactive yet

layout

Row(background='#efefef')
    [0] Column
        [0] Row(margin=(0, 25, 0, 25))
            [0] Markdown(str, sizing_mode='stretch_width')
        [1] Row(margin=(0, 25, 25, 25))
            [0] Column(margin=(0, 25, 0, 0))
                [0] Markdown(str)
                [1] Markdown(str)
                [2] Column
                    [0] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=1.16249875074925, name='u1', start=-1.28824507127872, step=0, value=(-0.98, 0.92), value_end=0.92, value_start=-0.98)
                    [1] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=1.78506223076923, name='u2', start=-1.46389651348651, step=0, value=(-0.83, 1.52), value_end=1.52, value_start=-0.83)
                    [2] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.55689243995005, name='u3', start=-3.36636916083916, step=0, value=(-3, 0.56), value_end=0.56, value_start=-3)
                    [3] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=3.54838884115884, name='u4', start=-0.591422778311688, step=0, value=(-0.59, 1.81), value_end=1.81, value_start=-0.59)
                [3] Markdown(str)
                [4] Column
                    [0] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=422.822, name='Molar Mass', start=18.015, step=0, value=(18, 204), value_end=204, value_start=18)
                    [1] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(name='Relative Polarity', step=0, value=(0, 0.59), value_end=0.59)
                    [2] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.111018595614765, name='H-Bond Acceptors', step=0, value=(0.0, 0.111018595614765), value_end=0.111018595614765)
                    [3] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.111018595614765, name='H-Bond Donors', step=0, value=(0.0, 0.111018595614765), value_end=0.111018595614765)
                    [4] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.967741935483871, name='projection quality', start=0.09859154929577466, step=0, value=(0.09859154929577466, ..., value_end=0.967741935483871, value_start=0.09859154929577466)
                    [5] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.967741935483871, name='projection quality', start=0.09859154929577466, step=0, value=(0.09859154929577466, ..., value_end=0.967741935483871, value_start=0.09859154929577466)
            [1] Column
                [0] Markdown(str, sizing_mode='stretch_width')
                [1] Bokeh(Figure)
                [2] Row
                    [0] Select(options=['None', 'u1', ...], value='None')
                    [1] Row(sizing_mode='stretch_width')
                        [0] Str(None, background='#2b83ba', height=30, sizing_mode='fixed', width=20)
                        [1] Markdown(str)
                        [2] Str(None, background='#afcc50', height=30, sizing_mode='fixed', width=20)
                        [3] Markdown(str)
                        [4] Str(None, background='#ffe05c', height=30, sizing_mode='fixed', width=20)
                        [5] Markdown(str)
                        [6] Str(None, background='#f5a60a', height=30, sizing_mode='fixed', width=20)
                        [7] Markdown(str)
                        [8] Str(None, background='#d7191c', height=30, sizing_mode='fixed', width=20)
                        [9] Markdown(str)
    [1] GridBox(margin=(25, 25, 0, 0), ncols=3, sizing_mode='stretch_both')

## Callbacks

Callbacks for **slider interactions**

In [56]:
visible = [False]*len(selected_var)
mapping = {v: k for k, v in dict(enumerate(selected_var)).items()}

        
def onSliderChanged(event):
    '''Actions upon attribute slider change.
    
    Attributes
    ----------
    event: bokeh.Events.Event
        information about the event that triggered the callback
    '''

    var = event.obj.name
    v_range = event.obj.value
        
    # if changed variable is currently displayed
    if var == layout[0][1][1][2][0].value:
        setColoring(var, v_range)
        
    # find the matching chart and update it
    for col in layout[1]:
        if col.name == var:
            df_polys, df_scatter, bounds, cnt_in, cnt_out = rangeset.compute_contours(var, v_range, bins=20 if col.name == 'groups' else 5)
            p,histo = _make_chart(var, df_polys, df_scatter, bounds, cnt_in, cnt_out)
            col[0].object = p
            col[1].object = histo

def onSliderChanged_released(event):
    '''Actions upon attribute slider change.
    
    Attributes
    ----------
    event: bokeh.Events.Event
        information about the event that triggered the callback
    '''

    var = event.obj.name
    v_range = event.obj.value
    
    print('\''+var+'\': ('+str(v_range[0])+','+str(v_range[1])+')')


def onAttributeSelected(event):
    '''Actions upon attribute checkbox change.
    
    Attributes
    ----------
    event: bokeh.Events.Event
        information about the event that triggered the callback
    '''
    var = event.obj.variable
    i = mapping[var]
    
    if event.obj.value == True:
        v_range = event.obj.slider.value
        
        df_polys, df_scatter, bounds, cnt_in, cnt_out = rangeset.compute_contours(var, v_range)
        p,p_histo = _make_chart(var, df_polys, df_scatter, bounds, cnt_in, cnt_out)
        pos_insert = sum(visible[:i])
        layout[1].insert(pos_insert, pn.Column(p,pn.panel(p_histo), name=var, margin=5))
    else:
        pos_remove = sum(visible[:i])
        layout[1].pop(pos_remove)
    
    visible[i] = event.obj.value  

# link widgets to their callbacks
for var in sliders.keys():
    sliders[var][0].param.watch(onAttributeSelected, 'value')
    sliders[var][1].param.watch(onSliderChanged, 'value')
    sliders[var][1].param.watch(onSliderChanged_released, 'value_throttled')

Callbacks **rangeset selection** in overview plot

In [57]:
def clearColoring():
    '''Remove rangeset augmentation from the embedding.'''
    
    global overview
    overview.legend.visible = False
    
    for r in overview.renderers:
        if r.name is not None and ('poly' in r.name or 'scatter' in r.name):
            r.visible = False
            r.muted = True
    
def setColoring(var, v_range=None):
    '''Compute and render the rangeset for a selected variable.
    
    Attributes
    ----------
    var: str
        the selected variable
    v_range: tuple (min,max)
        the user define value range for the rangeset
    '''
    
    global overview    
    overview.legend.visible = True
    
    if var == 'Primary Group':
        df_polys, df_scatter, bounds, cnt,cnt = rangeset.compute_contours('Übergruppe (ÜG)', val_range=top20_classes)
        for r in overview.renderers:
            if r.name is not None and ('poly' in r.name or 'scatter' in r.name):
                r.visible = False
                r.muted = True
    else:
        df_polys, df_scatter, bounds, cnt,cnt = rangeset.compute_contours(var, val_range=v_range, bins=bins)
        for r in overview.renderers:
            if r.name is not None and ('poly' in r.name or 'scatter' in r.name):
                r.visible = False
                r.muted = True

    if len(df_polys) > 0:
        for k in list(rangeset.labels.keys())[::-1]:
            label_id = rangeset.color2label(k)
            label = label_id
            
            g = df_polys[df_polys.color == k]
            r = overview.select('poly '+label)

            if len(r) > 0:
                r[0].visible = True
                r[0].muted = False
                r[0].data_source.data = dict(ColumnDataSource(g).data)
            else:
                overview.multi_polygons(source = g, xs='xs', ys='ys', name='poly '+label,
                                        color='color', alpha=.5, legend_label=label,
                                        line_color=None, muted_color='gray', muted_alpha=.1) 
                
        for k in list(rangeset.labels.keys())[::-1]:
            label_id = rangeset.color2label(k)
            label = label_id

            g = df_scatter[df_scatter.color == k]
            r = overview.select('scatter '+label)
            
            if len(r) > 0:
                r[0].visible = True
                r[0].muted = False
                r[0].data_source.data = dict(ColumnDataSource(g).data)
            else:
                overview.circle(source = g, x='x', y='y', size='size', name='scatter '+label,
                                color='color', alpha=1, legend_label=label,
                                muted_color='gray', muted_alpha=0) 

def onChangeColoring(event):
    '''Actions upon change of the rangeset attribute.
    
    Attributes
    ----------
    event: bokeh.Events.Event
        information about the event that triggered the callback
    '''
    var = event.obj.value
    
    if var == 'None':
        clearColoring()
    elif var == 'Primary Group':
        setColoring(var)
    else:
        v_range = sliders[var][1].value
        setColoring(var, v_range)
        
selectColoring.param.watch( onChangeColoring, 'value' )

Watcher(inst=Select(options=['None', 'u1', ...], value='None'), cls=<class 'panel.widgets.select.Select'>, fn=<function onChangeColoring at 0x7f6297d51d30>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False)

User **selection of data points** in the overview chart.

In [58]:
def onSelectionChanged(event):
    if event.final:
        sel_pp = pp[list(overview.select('points').data_source.selected.indices)]
        if len(sel_pp) == 0:
            source_selection.data = dict({'x': [], 'y': []})
        else:
            points = MultiPoint(sel_pp)
            poly = unary_union([polygon for polygon in triangulate(points) if rangeset._max_edge(polygon) < 5]).boundary.parallel_offset(-0.05).coords.xy
            source_selection.data = dict({'x': poly[0].tolist(), 'y': poly[1].tolist()})

overview.on_event(SelectionGeometry, onSelectionChanged)

In [59]:
layout.servable('NoLies')

Traceback (most recent call last):
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/pyviz_comms/__init__.py", line 325, in _handle_msg
 self._on_msg(msg)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/viewable.py", line 273, in _on_msg
 doc.unhold()
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/document/document.py", line 669, in unhold
 self._trigger_on_change(event)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/document/document.py", line 1180, in _trigger_on_change
 self._with_self_as_curdoc(event.callback_invoker)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/document/document.py", line 1198, in _with_self_as_curdoc
 return f()
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/util/callback_manager.py", line 161, in invoke
 callback(attr, old, new)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/reactive.py", line 301, in _comm_change
 self._process_events({attr: new})
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/reactive.py", line 262, in _process_events
 self.param.set_param(**self_events)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 1526, in set_param
 self_._batch_call_watchers()
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 1665, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 1627, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/tmp/ipykernel_41887/933570456.py", line 61, in onAttributeSelected
 layout[1].insert(pos_insert, pn.Column(p,pn.panel(p_histo), name=var, margin=5))
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/layout/base.py", line 307, in insert
 self.objects = new_objects
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 316, in _f
 instance_param.__set__(obj, val)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 318, in _f
 return f(self, obj, val)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 984, in __set__
 obj.param._call_watcher(watcher, event)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 1645, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/param/parameterized.py", line 1627, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/reactive.py", line 252, in _param_change
 self._apply_update(events, msg, model, ref)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/reactive.py", line 206, in _apply_update
 self._update_model(events, msg, root, model, doc, comm)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/layout/grid.py", line 160, in _update_model
 super(Panel, self)._update_model(events, msg, root, model, doc, comm)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/panel/reactive.py", line 219, in _update_model
 model.update(**msg)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/core/has_props.py", line 441, in update
 setattr(self, k, v)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/core/has_props.py", line 298, in __setattr__
 super().__setattr__(name, value)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/core/property/descriptors.py", line 552, in __set__
 self._internal_set(obj, value, setter=setter)
 File "/home/sohns/anaconda3/envs/nolies/lib/python3.8/site-packages/bokeh/core/property/descriptors.py", line 786, in _int

Row(background='#efefef')
    [0] Column
        [0] Row(margin=(0, 25, 0, 25))
            [0] Markdown(str, sizing_mode='stretch_width')
        [1] Row(margin=(0, 25, 25, 25))
            [0] Column(margin=(0, 25, 0, 0))
                [0] Markdown(str)
                [1] Markdown(str)
                [2] Column
                    [0] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=1.16249875074925, name='u1', start=-1.28824507127872, step=0, value=(-0.98, 0.92), value_end=0.92, value_start=-0.98)
                    [1] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=1.78506223076923, name='u2', start=-1.46389651348651, step=0, value=(-0.83, 1.52), value_end=1.52, value_start=-0.83)
                    [2] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.55689243995005, name='u3', start=-3.36636916083916, step=0, value=(-3, 0.56), value_end=0.56, value_start=-3)
                    [3] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=3.54838884115884, name='u4', start=-0.591422778311688, step=0, value=(-0.59, 1.81), value_end=1.81, value_start=-0.59)
                [3] Markdown(str)
                [4] Column
                    [0] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=422.822, name='Molar Mass', start=18.015, step=0, value=(18, 204), value_end=204, value_start=18)
                    [1] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(name='Relative Polarity', step=0, value=(0, 0.59), value_end=0.59)
                    [2] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.111018595614765, name='H-Bond Acceptors', step=0, value=(0.0, 0.111018595614765), value_end=0.111018595614765)
                    [3] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.111018595614765, name='H-Bond Donors', step=0, value=(0.0, 0.111018595614765), value_end=0.111018595614765)
                    [4] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.967741935483871, name='projection quality', start=0.09859154929577466, step=0, value=(0.09859154929577466, ..., value_end=0.967741935483871, value_start=0.09859154929577466)
                    [5] Row
                        [0] MyCheckbox(width=20)
                        [1] RangeSlider(end=0.967741935483871, name='projection quality', start=0.09859154929577466, step=0, value=(0.09859154929577466, ..., value_end=0.967741935483871, value_start=0.09859154929577466)
            [1] Column
                [0] Markdown(str, sizing_mode='stretch_width')
                [1] Bokeh(Figure)
                [2] Row
                    [0] Select(options=['None', 'u1', ...], value='None')
                    [1] Row(sizing_mode='stretch_width')
                        [0] Str(None, background='#2b83ba', height=30, sizing_mode='fixed', width=20)
                        [1] Markdown(str)
                        [2] Str(None, background='#afcc50', height=30, sizing_mode='fixed', width=20)
                        [3] Markdown(str)
                        [4] Str(None, background='#ffe05c', height=30, sizing_mode='fixed', width=20)
                        [5] Markdown(str)
                        [6] Str(None, background='#f5a60a', height=30, sizing_mode='fixed', width=20)
                        [7] Markdown(str)
                        [8] Str(None, background='#d7191c', height=30, sizing_mode='fixed', width=20)
                        [9] Markdown(str)
    [1] GridBox(margin=(25, 25, 0, 0), ncols=3, sizing_mode='stretch_both')